In [1]:
import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres

from hpbandster.optimizers import BOHB as BOHB
import pickle

import numpy as np

In [2]:
import sys
sys.path.insert(0, '..')


import System as SEQ
%run rod_EQS.ipynb


---------------------- SCIANN 0.6.4.5 ---------------------- 
For details, check out our review paper and the documentation at: 
 +  "https://www.sciencedirect.com/science/article/pii/S0045782520307374", 
 +  "https://arxiv.org/abs/2005.08803", 
 +  "https://www.sciann.com". 

 Need support or would like to contribute, please join sciann`s slack group: 
 +  "https://join.slack.com/t/sciann/shared_invite/zt-ne1f5jlx-k_dY8RGo3ZreDXwz0f~CeA" 
 
TensorFlow Version: 2.3.0 
Python Version: 3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)] 



In [3]:
import Pinn_Worker

In [4]:
test_gridObj = SEQ.Grid(2,{'x':0,'t':1},[[[0,2],[0,10]]],10)
configspecs = {
    'denspt':[5,12],
    'numNeurons':[10,100],
    'numLayers':[2,50],
    'activator':['tanh','relu','softmax','sigmoid'],
    'loss':['mae','mse'],
    'optimizer':['Adam','RMSprop','SGD','Nadam','Ftrl'],
    'batch_size':[5000,10000,15000,25000]
}




In [5]:
def u_n(n,x,t):
  mu_n = (2*n+1)*np.pi/4
  T_n =  np.exp(-t*(mu_n**2))
  X_n = -np.cos(mu_n*x)*2.5/(mu_n**2)
  return X_n*T_n

depth = 100

u_pred_Simb = (np.sum((np.array([u_n(n,test_gridObj.grid[:,0],test_gridObj.grid[:,1]) for n in range(depth)])),axis=0)+5)[:,None]
valData = np.concatenate((test_gridObj.grid,u_pred_Simb),axis=1)

In [7]:
# Step 1: Start a nameserver
# Every run needs a nameserver. It could be a 'static' server with a
# permanent address, but here it will be started for the local machine with the default port.
# The nameserver manages the concurrent running workers across all possible threads or clusternodes.
# Note the run_id argument. This uniquely identifies a run of any HpBandSter optimizer.
NS = hpns.NameServer(run_id='rod_GridSearch_1', host='127.0.0.1', port=None)
NS.start()

# Step 2: Start a worker
# Now we can instantiate a worker, providing the mandatory information
# Besides the sleep_interval, we need to define the nameserver information and
# the same run_id as above. After that, we can start the worker in the background,
# where it will wait for incoming configurations to evaluate.

numWorkers = 5
experiment_name = 'chandrasekhar'

workers=[]
for i in range(1,numWorkers+1):
    
    w = Pinn_Worker.PINN_Worker(
    valData = valData,
    test_gridObj=test_gridObj,
    PDESystem=mySys,
    configspecs = configspecs,
    valFromFEM=False,
    experiment_name = experiment_name,
    nameserver='127.0.0.1',
    run_id='rod_GridSearch_1',id=i)
    
    w.run(background=False)
    
    workers.append(w)




22:52:03 WORKER: Connected to nameserver <Pyro4.core.Proxy at 0x285fe082c48; connected IPv4; for PYRO:Pyro.NameServer@127.0.0.1:9090>
22:52:03 WORKER: No dispatcher found. Waiting for one to initiate contact.
22:52:03 WORKER: start listening for jobs
22:52:03 WORKER: Connected to nameserver <Pyro4.core.Proxy at 0x285fe0a7308; connected IPv4; for PYRO:Pyro.NameServer@127.0.0.1:9090>
22:52:03 WORKER: No dispatcher found. Waiting for one to initiate contact.
22:52:03 WORKER: start listening for jobs
22:52:03 WORKER: Connected to nameserver <Pyro4.core.Proxy at 0x285fe0afbc8; connected IPv4; for PYRO:Pyro.NameServer@127.0.0.1:9090>
22:52:03 WORKER: No dispatcher found. Waiting for one to initiate contact.
22:52:03 WORKER: start listening for jobs
22:52:03 WORKER: Connected to nameserver <Pyro4.core.Proxy at 0x285fe0ba488; connected IPv4; for PYRO:Pyro.NameServer@127.0.0.1:9090>
22:52:03 WORKER: No dispatcher found. Waiting for one to initiate contact.
22:52:03 WORKER: start listening for j

Applying validation data to test grid
Saving PDE and config
Worker ready
Applying validation data to test grid
Saving PDE and config
Worker ready
Applying validation data to test grid
Saving PDE and config
Worker ready
Applying validation data to test grid
Saving PDE and config
Worker ready
Applying validation data to test grid
Saving PDE and config
Worker ready


22:52:03 WORKER: Connected to nameserver <Pyro4.core.Proxy at 0x285fe0c3248; connected IPv4; for PYRO:Pyro.NameServer@127.0.0.1:9090>
22:52:03 WORKER: No dispatcher found. Waiting for one to initiate contact.
22:52:03 WORKER: start listening for jobs


In [ ]:
# bohb = BOHB(  configspace = w.get_configspace(),
#               run_id = 'example1', nameserver='127.0.0.1',
#               min_budget=10, max_budget=1000
#            )
# res = bohb.run(n_iterations=25
#               )

bohb = BOHB(  configspace = w.get_configspace(),
              run_id = 'rod_GridSearch_1',
              min_budget=100, max_budget=5000
           )
res = bohb.run(n_iterations=15, min_n_workers=numWorkers)


In [ ]:
import tensorflow as tf
tf.config.list_logical_devices()